# executorlib

https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

## Define workflow with executorlib

In [1]:
import os

In [2]:
from workflow import generate_mesh, convert_to_xdmf, poisson, plot_over_line, substitute_macros, compile_paper 

In [3]:
from executorlib import SingleNodeExecutor, get_item_from_future

In [5]:
domain_size = 2.0

In [6]:
source_directory = os.path.abspath(os.path.join(os.curdir, "source"))

In [ ]:
workflow_json_filename = "executorlib_nfdi.json"

In [7]:
with SingleNodeExecutor(export_workflow_filename=workflow_json_filename) as exe:
    gmsh_output_file = exe.submit(
        generate_mesh,
        domain_size=domain_size,
        source_directory=source_directory,
    )
    meshio_output_dict = exe.submit(
        convert_to_xdmf,
        gmsh_output_file=gmsh_output_file,
    )
    poisson_dict = exe.submit(
        poisson,
        meshio_output_xdmf=get_item_from_future(meshio_output_dict, key="xdmf_file"), 
        meshio_output_h5=get_item_from_future(meshio_output_dict, key="h5_file"),
        source_directory=source_directory,
    )
    pvbatch_output_file = exe.submit(
        plot_over_line,
        poisson_output_pvd_file=get_item_from_future(poisson_dict, key="pvd_file"), 
        poisson_output_vtu_file=get_item_from_future(poisson_dict, key="vtu_file"),
        source_directory=source_directory,
    )
    macros_tex_file = exe.submit(
        substitute_macros,
        pvbatch_output_file=pvbatch_output_file, 
        ndofs=get_item_from_future(poisson_dict, key="numdofs"), 
        domain_size=domain_size,
        source_directory=source_directory,
    )
    paper_output = exe.submit(
        compile_paper,
        macros_tex=macros_tex_file, 
        plot_file=pvbatch_output_file,
        source_directory=source_directory,
    )

## Load Workflow with aiida

In [15]:
from aiida import load_profile

load_profile()

Profile<uuid='3ce302a995914d2e8a7b6327a10fe381' name='pwd'>

In [16]:
from python_workflow_definition.aiida import load_workflow_json

In [17]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [18]:
wg.run()

05/24/2025 05:45:10 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_mesh6
05/24/2025 05:45:12 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGraphEngine|update_task_state]: Task: generate_mesh6, type: PyFunction, finished.
05/24/2025 05:45:12 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGraphEngine|continue_workgraph]: tasks ready to run: convert_to_xdmf5
05/24/2025 05:45:13 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGraphEngine|update_task_state]: Task: convert_to_xdmf5, type: PyFunction, finished.
05/24/2025 05:45:13 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGraphEngine|continue_workgraph]: tasks ready to run: poisson4
05/24/2025 05:45:16 AM <4326> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [81|WorkGrap

## Load Workflow with jobflow

In [19]:
from python_workflow_definition.jobflow import load_workflow_json

In [20]:
from jobflow.managers.local import run_locally

In [21]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [22]:
result = run_locally(flow)
result

2025-05-24 05:45:23,211 INFO Started executing jobs locally
2025-05-24 05:45:23,487 INFO Starting job - generate_mesh (afd8b83e-2283-40e8-9a75-4553a7ca72d0)
2025-05-24 05:45:24,604 INFO Finished job - generate_mesh (afd8b83e-2283-40e8-9a75-4553a7ca72d0)
2025-05-24 05:45:24,605 INFO Starting job - convert_to_xdmf (6723daca-18b6-4f38-8b3e-a92c2a227dc3)
2025-05-24 05:45:26,001 INFO Finished job - convert_to_xdmf (6723daca-18b6-4f38-8b3e-a92c2a227dc3)
2025-05-24 05:45:26,002 INFO Starting job - poisson (db43f907-45eb-4596-ac6b-ab5978cc5e17)
2025-05-24 05:45:28,507 INFO Finished job - poisson (db43f907-45eb-4596-ac6b-ab5978cc5e17)
2025-05-24 05:45:28,509 INFO Starting job - plot_over_line (b6493a73-f134-4067-ba0d-d6e2cd246afe)
2025-05-24 05:45:29,933 INFO Finished job - plot_over_line (b6493a73-f134-4067-ba0d-d6e2cd246afe)
2025-05-24 05:45:29,934 INFO Starting job - substitute_macros (65c3db44-dcc1-44ca-a1bb-374e8f5c771e)
2025-05-24 05:45:30,774 INFO Finished job - substitute_macros (65c3db

{'afd8b83e-2283-40e8-9a75-4553a7ca72d0': {1: Response(output='/home/jovyan/example_workflows/nfdi/preprocessing/square.msh', detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan/example_workflows/nfdi'))},
 '6723daca-18b6-4f38-8b3e-a92c2a227dc3': {1: Response(output={'xdmf_file': '/home/jovyan/example_workflows/nfdi/preprocessing/square.xdmf', 'h5_file': '/home/jovyan/example_workflows/nfdi/preprocessing/square.h5'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan/example_workflows/nfdi'))},
 'db43f907-45eb-4596-ac6b-ab5978cc5e17': {1: Response(output={'numdofs': 357, 'pvd_file': '/home/jovyan/example_workflows/nfdi/processing/poisson.pvd', 'vtu_file': '/home/jovyan/example_workflows/nfdi/processing/poisson000000.vtu'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_

# Load Workflow with pyiron_base

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
delayed_object_lst[-1].draw()

In [ ]:
delayed_object_lst[-1].pull()

## Load Workflow with pyiron_workflow

In [ ]:
from python_workflow_definition.pyiron_workflow import load_workflow_json

In [ ]:
wf = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
wf.draw(size=(10,10))

In [ ]:
wf.run()